In [88]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.ie.options import Options
from selenium.webdriver.support.ui import Select,WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import requests

import pymysql
from datetime import datetime, timedelta
from google_trans_new import google_translator
import time
import random

host = '10.55.23.101'
port = 33060
user = 'root'
passwd = "1234"
db = 'idap'

showPrintMSG = 1 # 0:不呈現，1:呈現 debug mode 
web = 'reuters'
language = 'en'
sleep_sec = 2
lang_src=language
lang_tgt='zh-tw'

def gl_translator(text, lang_src, lang_tgt) :
    url_suffix_list = ['ac','ad','ae','al','am','as','at','az','ba','be','bf','bg','bi','bj','bs','bt','by','ca','cat','cc','cd','cf','cg','ch','ci','cl','cm','cn','co.ao','co.bw','co.ck','co.cr','co.id','co.il','co.in','co.jp','co.ke','co.kr','co.ls','co.ma','co.mz','co.nz','co.th','co.tz','co.ug','co.uk','co.uz','co.ve','co.vi','co.za','co.zm','co.zw','co','com.af','com.ag','com.ai','com.ar','com.au','com.bd','com.bh','com.bn','com.bo','com.br','com.bz','com.co','com.cu','com.cy','com.do','com.ec','com.eg','com.et','com.fj','com.gh','com.gi','com.gt','com.hk','com.jm','com.kh','com.kw','com.lb','com.lc','com.ly','com.mm','com.mt','com.mx','com.my','com.na','com.ng','com.ni','com.np','com.om','com.pa','com.pe','com.pg','com.ph','com.pk','com.pr','com.py','com.qa','com.sa','com.sb','com.sg','com.sl','com.sv','com.tj','com.tr','com.tw','com.ua','com.uy','com.vc','com.vn','com','cv','cx','cz','de','dj','dk','dm','dz','ee','es','eu','fi','fm','fr','ga','ge','gf','gg','gl','gm','gp','gr','gy','hn','hr','ht','hu','ie','im','io','iq','is','it','je','jo','kg','ki','kz','la','li','lk','lt','lu','lv','md','me','mg','mk','ml','mn','ms','mu','mv','mw','ne','nf','nl','no','nr','nu','pl','pn','ps','pt','ro','rs','ru','rw','sc','se','sh','si','sk','sm','sn','so','sr','st','td','tg','tk','tl','tm','tn','to','tt','us','vg','vu','ws']
    isSuccess = False
    url_suffix_inx = random.randint(0, len(url_suffix_list)-1)
    content_tw = ''
    while isSuccess == False :
        try :
            translator = google_translator(url_suffix=url_suffix_list[url_suffix_inx])        
            content_tw = translator.translate(text, lang_src=lang_src, lang_tgt=lang_tgt)
            isSuccess = True
        except Exception as e:
            url_suffix_inx = random.randint(0, len(url_suffix_list)-1)
    return content_tw

def reuters_1(archiveDate = 10) :
    baseUrl = 'https://www.reuters.com/'
    category = ['business','technology']
    option = webdriver.ChromeOptions()
    driver = webdriver.Chrome(r"D:\chromedriver.exe",chrome_options=option)
    for c in range(0,len(category)) :
        targetUrl = baseUrl+category[c]
        print(category[c])
        driver.get(targetUrl)
        time.sleep(8)
        deleteFromDate = datetime.strftime(datetime.now() - timedelta(archiveDate), '%Y%m%d')
        conn = pymysql.connect(host=host, port=port, user=user, passwd=passwd, db=db)
        cur = conn.cursor()

        tag = 'industry'
        creationdate = datetime.now()

        for i in range(1,4) :    
            url = driver.find_element_by_xpath('//*[@id="main-content"]/div[2]/div/div[1]/div/div/div['+str(i)+']/a').get_attribute('href')
            Tmp = driver.find_element_by_xpath('//*[@id="main-content"]/div[2]/div/div[1]/div/div/div['+str(i)+']/a').text
            title = Tmp.split('\n')[1]
            if i == 1 :
                publishdate = datetime.strftime(datetime.now(), '%Y%m%d')
            else :
                try :
                    publishdate = ':' in Tmp.split('\n')[2] and datetime.strftime(datetime.now(), '%Y%m%d') or datetime.strftime(datetime.strptime(Tmp.split('\n')[2],'%B %d, %Y'),'%Y%m%d')
                except Exception as e :           
                    publishdate = ':' in Tmp.split('\n')[3] and datetime.strftime(datetime.now(), '%Y%m%d') or datetime.strftime(datetime.strptime(Tmp.split('\n')[3],'%B %d, %Y'),'%Y%m%d')
            print(i,title,publishdate,url)        
            if int(publishdate) > int(deleteFromDate) :
                cur.execute('select count(1) from news_daily_source where url=%s',(url))
                if cur.fetchone()[0] == 0 :
                    title_tw = gl_translator(title, lang_src, lang_tgt)
                    cur.execute('insert into news_daily_source(web, title, title_tw, publishdate, url, creationdate,language,tag) SELECT * FROM (SELECT %s, %s, %s, %s, %s, %s, %s, %s) AS tmp WHERE NOT EXISTS (SELECT url FROM news_daily_source WHERE url = %s) LIMIT 1',(web, title, title_tw, publishdate, url, creationdate,language,tag, url))
                    cur.execute('commit') 

        for i in range(1,9) :
            url = driver.find_element_by_xpath('//*[@id="main-content"]/div[3]/div/div['+str(i)+']/a').get_attribute('href')
            Tmp = driver.find_element_by_xpath('//*[@id="main-content"]/div[3]/div/div['+str(i)+']/a').text 
            #tag = Tmp.split('\n')[0]
            title = Tmp.split('\n')[1]
            try :
                publishdate = ':' in Tmp.split('\n')[2] and datetime.strftime(datetime.now(), '%Y%m%d') or datetime.strftime(datetime.strptime(Tmp.split('\n')[2],'%B %d, %Y'),'%Y%m%d')
            except Exception as e :            
                publishdate = ':' in Tmp.split('\n')[3] and datetime.strftime(datetime.now(), '%Y%m%d') or datetime.strftime(datetime.strptime(Tmp.split('\n')[3],'%B %d, %Y'),'%Y%m%d')
            print(i,title,publishdate,url)
            if int(publishdate) > int(deleteFromDate) :
                cur.execute('select count(1) from news_daily_source where url=%s',(url))
                if cur.fetchone()[0] == 0 :
                    title_tw = gl_translator(title, lang_src, lang_tgt)
                    cur.execute('insert into news_daily_source(web, title, title_tw, publishdate, url, creationdate,language,tag) SELECT * FROM (SELECT %s, %s, %s, %s, %s, %s, %s, %s) AS tmp WHERE NOT EXISTS (SELECT url FROM news_daily_source WHERE url = %s) LIMIT 1',(web, title, title_tw, publishdate, url, creationdate,language,tag, url))
                    cur.execute('commit') 

        for i in range(1,10) :    
            url = driver.find_element_by_xpath('//*[@id="main-content"]/div[4]/div/div[1]/div/div[1]/div['+str(i)+']/a').get_attribute('href')
            Tmp = driver.find_element_by_xpath('//*[@id="main-content"]/div[4]/div/div[1]/div/div[1]/div['+str(i)+']/a').text
            title = Tmp.split('\n')[1]
            try :
                publishdate = ':' in Tmp.split('\n')[2] and datetime.strftime(datetime.now(), '%Y%m%d') or datetime.strftime(datetime.strptime(Tmp.split('\n')[2],'%B %d, %Y'),'%Y%m%d')
            except Exception as e :          
                publishdate = ':' in Tmp.split('\n')[3] and datetime.strftime(datetime.now(), '%Y%m%d') or datetime.strftime(datetime.strptime(Tmp.split('\n')[3],'%B %d, %Y'),'%Y%m%d')
            print(i,title,publishdate,url)
            if int(publishdate) > int(deleteFromDate) :
                cur.execute('select count(1) from news_daily_source where url=%s',(url))
                if cur.fetchone()[0] == 0 :
                    title_tw = gl_translator(title, lang_src, lang_tgt)
                    cur.execute('insert into news_daily_source(web, title, title_tw, publishdate, url, creationdate,language,tag) SELECT * FROM (SELECT %s, %s, %s, %s, %s, %s, %s, %s) AS tmp WHERE NOT EXISTS (SELECT url FROM news_daily_source WHERE url = %s) LIMIT 1',(web, title, title_tw, publishdate, url, creationdate,language,tag, url))
                    cur.execute('commit') 
    cur.close()
    conn.close()
    driver.close()

def reuters_2() :
    conn = pymysql.connect(host=host, port=port, user=user, passwd=passwd, db=db)
    cur = conn.cursor()
    cur1 = conn.cursor()
    
    cur.execute('select count(1) from news_daily_source where status=0 and content_tw is null and web="reuters" order by id')
    count = cur.fetchone()[0]
    if showPrintMSG :
        print('count for processing : ',count)
    if count > 0 :
        c=1
        cur.execute('select url from news_daily_source where status=0 and content_tw is null and web="reuters" order by id')
        for r in cur :
            url = r[0]
            if showPrintMSG :
                print(c,'/',count,'. ',url)
            res = requests.get(url)
            res.encoding = 'utf-8'
            soup = BeautifulSoup(res.text, 'html.parser')        
            content = soup.select("article > div > div > div")[0].text
            if len(content) > 4900 : 
                conetntTmp = ''
                content_tw = ''
                subContentList = content.split('. ')
                for i in range(0,len(subContentList)):
                    conetntTmp = conetntTmp+subContentList[i]+' '
                    if len(conetntTmp) > 3000 :
                        content_tw = content_tw + gl_translator(conetntTmp, lang_src, lang_tgt)+' '
                        conetntTmp = ''
                content_tw = content_tw + gl_translator(conetntTmp, lang_src, lang_tgt)+' '        
            else :
                content_tw = gl_translator(content, lang_src, lang_tgt)
            cur1.execute('update news_daily_source set content=%s,content_tw=%s where url=%s',(content,content_tw,url))
            cur1.execute('commit')
            #if not showPrintMSG :
            time.sleep(sleep_sec)
            c = c + 1
    cur.close()
    conn.close() 
    cur1.close()            

def reuters_etl() : 
    reuters_1()
    reuters_2()         

In [112]:
reuters_2()


count for processing :  5
1 / 5 .  https://www.reuters.com/business/energy/frustrated-canada-presses-white-house-keep-great-lakes-oil-pipeline-open-2021-04-26/
2 / 5 .  https://www.reuters.com/technology/analysis-china-digital-currency-trials-show-threat-alipay-wechat-duopoly-2021-04-26/
3 / 5 .  https://www.reuters.com/business/us-manufacturing-activity-accelerates-early-april-supply-constraints-worsening-2021-04-23/
4 / 5 .  https://www.reuters.com/technology/wall-street-sees-long-road-ahead-intel-seeks-regain-market-share-2021-04-23/
5 / 5 .  https://www.reuters.com/business/aerospace-defense/spacex-rocketship-launches-4-astronauts-nasa-mission-space-station-2021-04-23/


In [100]:
conn = pymysql.connect(host=host, port=port, user=user, passwd=passwd, db=db)
cur = conn.cursor()


cur.execute('SELECT content FROM `news_daily_source` WHERE id=11451')
for r in cur :
    content = r[0]
    print(len(content))


6180


In [104]:
conetntTmp = ''
subContentList = content.split('. ')

for i in range(0,len(subContentList)):
    #conetntTmp = conetntTmp+subContentList[i]+' '
    #if(conetntTmp)
    print(subContentList[i])
    print(len(subContentList[i]),'\n ')
    

1/5A signpost marks the presence of Enbridge's Line 5 pipeline, which Michigan Governor Gretchen Whitmer ordered shut down in May 2021, in Sarnia, Ontario, Canada March 20, 2021
177 
 
Picture taken March 20, 2021
28 
 
REUTERS/Carlos Osorio/File PhotoRead MoreCanada is pushing on several diplomatic fronts against the U.S
103 
 
state of Michigan's efforts to close a cross-border oil pipeline, the second such dispute since Joe Biden became U.S
116 
 
president in January, complicating the governments' efforts to work together to lower carbon emissions.The conflict over the aging but key pipeline highlights the disruptions caused by a global shift away from fossil fuels
222 
 
Both governments are working to accelerate the energy transition, but their oil industries are interdependent, so a policy shift in one country can affect energy supply, and the political balance, in the other.The United States imports more crude from Canada than any other nation, at about 3.7 million barrels per 

In [34]:
datetime.strftime('April 25, 2021', '%Y%m%d')

TypeError: descriptor 'strftime' requires a 'datetime.date' object but received a 'str'

In [7]:
url = driver.find_element_by_xpath('//*[@id="top-news"]/div/div[1]/div[1]/article[1]').get_attribute('href')
title = driver.find_element_by_xpath('//*[@id="top-news"]/div/div[1]/div[1]/article[1]').text
print(url,title)

None GameStop’s CEO Is Getting Millions on His Way Out
The rally in the videogame retailer’s share price has provided an unusual boost in the value of stock awards for departing executives.21
